In [26]:
%reload_ext autoreload
%autoreload 2

from Modules import DynamicSymbol, StaticSymbol, DynamicSymbols, StaticSymbols, diff_t, Drehmatrix, DynamicSystem, StaticSystem, disp
import symengine as se
import sympy

se.init_printing()
sympy.init_printing(use_latex='mathjax')

In [27]:
[iG,c,s,m,CA,CM,km] = StaticSymbols(["i_G","c","s","m","C_A","C_M","k_m"])
params = [iG,c,s,m,CA,CM,km]

In [28]:
# q_var = DynamicSymbols(["q_M","q_A"],1,2,True)
q_var = DynamicSymbol("q",2,2)
u_var = DynamicSymbol("M_{Mot}",1,0)

vars = DynamicSymbols(["q_M","q_A"],1,2)
[q,q_dot,q_ddot] = vars
[q,q_dot,q_ddot] = [se.Matrix(q),se.Matrix(q_dot),se.Matrix(q_ddot)]
[[qM,qA],[qM_dot, qA_dot],[qM_ddot, qA_ddot]] = vars


u = u_var.vars
#[q,q_dot,q_ddot] = q_var.vars
#[[qM,qA],[qM_dot, qA_dot],[qM_ddot, qA_ddot]] = q_var.vars
#display(q_var, u_var)
M_Mot = u[0]

In [29]:
K_J_AS = se.diag(0,0,CA)
K_J_AS

[0, 0, 0]
[0, 0, 0]
[0, 0, C_{A}^{}]

In [30]:
K_J_MS = se.zeros(3)
K_J_MS[2,2] = CM


K_J_MS

[0, 0, 0]
[0, 0, 0]
[0, 0, C_{M}^{}]

In [31]:
R_IK = Drehmatrix(se.Matrix([0,0,qM]))
display(R_IK)

[cos(q_{M}^{}(t)), -sin(q_{M}^{}(t)), 0]
[sin(q_{M}^{}(t)), cos(q_{M}^{}(t)), 0]
[0, 0, 1]

In [32]:
K_r_s = se.Matrix([s,0,0])
K_r_s

[s_{}^{}]
[0]
[0]

In [33]:
K_w_K = se.Matrix([0,0,qA_dot+qM_dot])
K_w_K

[0]
[0]
[\dot{q}_{A}^{}(t) + \dot{q}_{M}^{}(t)]

In [34]:
K_w_M = se.Matrix([0,0,iG*qM_dot])
K_w_M

[0]
[0]
[\dot{q}_{M}^{}(t)*i_{G}^{}]

In [35]:
K_v_S = diff_t(K_r_s) + K_w_K.cross(K_r_s)
K_v_S

[0]
[(\dot{q}_{A}^{}(t) + \dot{q}_{M}^{}(t))*s_{}^{}]
[0]

In [36]:
K_p = m * K_v_S
K_p

[0]
[(\dot{q}_{A}^{}(t) + \dot{q}_{M}^{}(t))*m_{}^{}*s_{}^{}]
[0]

In [37]:
K_L_A = K_J_AS * K_w_K
K_L_A

[0]
[0]
[(\dot{q}_{A}^{}(t) + \dot{q}_{M}^{}(t))*C_{A}^{}]

In [38]:
K_L_M = K_J_MS * K_w_M
K_L_M

[0]
[0]
[\dot{q}_{M}^{}(t)*C_{M}^{}*i_{G}^{}]

In [39]:
K_Me_A = se.Matrix([0,0,-qA * c])
K_Me_A

[0]
[0]
[-c_{}^{}*q_{A}^{}(t)]

In [40]:
K_Me_M = se.Matrix([0,0,M_Mot]) - K_Me_A / iG
K_Me_M

[0]
[0]
[c_{}^{}*q_{A}^{}(t)/i_{G}^{} + M_{Mot}^{}(t)]

In [41]:
display(diff_t(K_p), K_p)
part1 = K_v_S.jacobian(q_dot).transpose() * (diff_t(K_p) + K_w_K.cross(K_p))

disp(part1)

[0]
[(\ddot{q}_{A}^{}(t) + \ddot{q}_{M}^{}(t))*m_{}^{}*s_{}^{}]
[0]

[0]
[(\dot{q}_{A}^{}(t) + \dot{q}_{M}^{}(t))*m_{}^{}*s_{}^{}]
[0]

⎡                 2                                            ⎤
⎢m_{}__{}⋅s_{}__{} ⋅(\ddot{q}_{A}__{}(t) + \ddot{q}_{M}__{}(t))⎥
⎢                                                              ⎥
⎢                 2                                            ⎥
⎣m_{}__{}⋅s_{}__{} ⋅(\ddot{q}_{A}__{}(t) + \ddot{q}_{M}__{}(t))⎦

In [42]:
part2 = K_w_K.jacobian(q_dot).transpose() * (diff_t(K_L_A) + K_w_K.cross(K_L_A) - K_Me_A)
part2

[c_{}^{}*q_{A}^{}(t) + (\ddot{q}_{A}^{}(t) + \ddot{q}_{M}^{}(t))*C_{A}^{}]
[c_{}^{}*q_{A}^{}(t) + (\ddot{q}_{A}^{}(t) + \ddot{q}_{M}^{}(t))*C_{A}^{}]

In [43]:
part3 = K_w_M.jacobian(q_dot).transpose() * (diff_t(K_L_M) + K_w_M.cross(K_L_M) - K_Me_M)
part3

[(\ddot{q}_{M}^{}(t)*C_{M}^{}*i_{G}^{} - (c_{}^{}*q_{A}^{}(t)/i_{G}^{} + M_{Mot}^{}(t)))*i_{G}^{}]
[0]

In [44]:
parts = part1 + part2 + part3
disp(parts)

⎡                                                                             
⎢C_{A}__{}⋅(\ddot{q}_{A}__{}(t) + \ddot{q}_{M}__{}(t)) + c_{}__{}⋅q_{A}__{}(t)
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣                                               C_{A}__{}⋅(\ddot{q}_{A}__{}(t)

             ⎛                                          c_{}__{}⋅q_{A}__{}(t) 
 + i_{G}__{}⋅⎜C_{M}__{}⋅i_{G}__{}⋅\ddot{q}_{M}__{}(t) - ───────────────────── 
             ⎝                                                i_{G}__{}       
                                                                              
                                                                   2          
 + \ddot{q}_{M}__{}(t)) + c_{}__{}⋅q_{A}__{}(t) + m_{}__{}⋅s_{}__{} ⋅(\ddot{q}

                ⎞                    2            

In [45]:
M = parts.jacobian(q_ddot)
disp(M)

⎡                               2                    2                        
⎢C_{A}__{} + C_{M}__{}⋅i_{G}__{}  + m_{}__{}⋅s_{}__{}   C_{A}__{} + m_{}__{}⋅s
⎢                                                                             
⎢                                        2                                    
⎣           C_{A}__{} + m_{}__{}⋅s_{}__{}               C_{A}__{} + m_{}__{}⋅s

       2⎤
_{}__{} ⎥
        ⎥
       2⎥
_{}__{} ⎦

In [46]:
g = parts - M @  q_ddot
g = g.expand()
g

[-M_{Mot}^{}(t)*i_{G}^{}]
[c_{}^{}*q_{A}^{}(t)]

In [47]:
f = q_dot.col_join((-M.inv() * g.expand()))

In [48]:
x_vars = q.col_join(q_dot)
h = se.Matrix([[qM + qA],[(qM_dot + qA_dot * qA)]])

dynSys = DynamicSystem(x_vars,u)
dynSys.addStateEquations(f)
dynSys.addOutput(h,"y")
dynSys.addOutput(h,"l")
dynSys.addInput(u,"u")
dynSys.addParameter(params)
#sys = Robotik_System(f,h,q_var,u_var,params,False)

In [49]:
[A,B,C,D] = dynSys.linearize(se.Matrix([0,0,0,0]),se.Matrix([0]))
disp(A,B,C,D)


⎡0                                                                         0  
⎢                                                                             
⎢0                                                                         0  
⎢                                                                             
⎢                                                               ⎛             
⎢                                                      c_{}__{}⋅⎝C_{A}__{} + m
⎢0  ──────────────────────────────────────────────────────────────────────────
⎢                                                           ⎛                 
⎢                                                           ⎜                 
⎢   ⎛                               2                    2⎞ ⎜                 
⎢   ⎝C_{A}__{} + C_{M}__{}⋅i_{G}__{}  + m_{}__{}⋅s_{}__{} ⎠⋅⎜C_{A}__{} + m_{}_
⎢                                                           ⎜                 
⎢                                                   

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢          ⎛                                                                  
⎢          ⎜                                                       ⎛          
⎢          ⎜                                                       ⎝C_{A}__{} 
⎢i_{G}__{}⋅⎜──────────────────────────────────────────────────────────────────
⎢          ⎜                                                        ⎛         
⎢          ⎜                                                        ⎜         
⎢          ⎜⎛                               2                    2⎞ ⎜         
⎢          ⎜⎝C_{A}__{} + C_{M}__{}⋅i_{G}__{}  + m_{}__{}⋅s_{}__{} ⎠⋅⎜C_{A}__{}
⎢          ⎜                                        

⎡1  1  0  0⎤
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎢1  1  0  0⎥
⎢          ⎥
⎣0  0  1  0⎦

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [50]:
dynSys.write_ABCD_to_File("ss_matrices")
dynSys.write_init_File("init_params")
dynSys.write_SFunction("sfun_test")
dynSys.write_MFunctions("Test")